In [71]:
import pandas as pd
import numpy as np

In [72]:
df_cpi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\cpi_index_VN.csv')
df_fdi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\fdi_inflow_VN.csv')
df_gdp = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\gdp_growth_VN.csv')
df_retail = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Retail_processed.csv.csv')
df_unemployment = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Unemployment_processed.csv')

In [73]:
import pandas as pd
import numpy as np

def prepare_data(df, date_col='date', value_col=None):
    """Đảm bảo cột ngày tháng ở định dạng datetime và thiết lập làm index."""
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    if value_col is None:
        value_col = df.columns[0]
    df = df[[value_col]].rename(columns={value_col: 'annual_value'})
    # ⭐ QUAN TRỌNG: Sort index theo thứ tự thời gian
    df = df.sort_index()
    return df

# Chuẩn bị dữ liệu
data_frames = {
    'fdi_inflow': prepare_data(df_fdi, value_col='fdi_inflow'),
    'cpi_index': prepare_data(df_cpi, value_col='cpi_index'),
    'gdp_growth': prepare_data(df_gdp, value_col='gdp_percent'),
    'retail_value': prepare_data(df_retail, value_col='retail_value'),
    'unemployment_value': prepare_data(df_unemployment, value_col='unemployment_value')
}

# Định nghĩa tham số dựa trên bản chất của biến số
DISAGG_PARAMS = {
    # Dòng chảy (Flow): Tổng 4 quý = giá trị năm
    'fdi_inflow': {'agg_func': 'sum'}, 
    # Tỷ lệ/Chỉ số (Rate/Index): Trung bình 4 quý = giá trị năm
    'cpi_index': {'agg_func': 'mean'},
    'gdp_growth': {'agg_func': 'mean'},
    'retail_value': {'agg_func': 'mean'},
    'unemployment_value': {'agg_func': 'mean'}
}

def benchmarked_linear_disaggregation(df_annual, agg_func):
    """
    Phân tách dữ liệu năm thành dữ liệu quý sử dụng phương pháp 
    nội suy tuyến tính được chuẩn hóa (benchmarking).
    """
    value_col = 'annual_value'
    
    # ⭐ Đảm bảo index được sort
    df_annual = df_annual.sort_index()
    
    print(f"  Index gốc (đầu): {df_annual.index[:3].tolist()}")
    print(f"  Index gốc (cuối): {df_annual.index[-3:].tolist()}")
    
    # 1. UPSAMPLING VÀ NỘI SUY CƠ SỞ
    # Chuyển về đầu năm (YYYY-01-01) để resample chính xác
    df_annual_start = df_annual.copy()
    df_annual_start.index = df_annual_start.index.to_period('Y').to_timestamp('YS')
    
    print(f"  Index sau to_timestamp('YS'): {df_annual_start.index[:3].tolist()}")
    
    # Resample thành quý bắt đầu và điền forward, sau đó nội suy
    df_quarterly_resampled = df_annual_start.resample('QS').asfreq()
    print(f"  Số hàng sau resample QS: {len(df_quarterly_resampled)}")
    print(f"  Index resample (đầu): {df_quarterly_resampled.index[:5].tolist()}")
    
    # Nội suy tuyến tính
    df_quarterly_interpolated = df_quarterly_resampled.interpolate(method='linear')
    df_quarterly_interpolated.columns = ['interpolated_value']
    
    print(f"  Số hàng sau interpolate: {len(df_quarterly_interpolated)}")
    print(f"  Giá trị NaN sau interpolate: {df_quarterly_interpolated['interpolated_value'].isna().sum()}")
    
    # 2. TÍNH TOÁN CHUẨN HÓA (Benchmarking)
    
    # Tạo key năm để map dữ liệu
    df_annual_temp = df_annual.copy()
    df_annual_temp['annual_period'] = df_annual_temp.index.to_period('Y')
    df_quarterly_interpolated['annual_period'] = df_quarterly_interpolated.index.to_period('Y')
    
    # Map giá trị năm gốc
    annual_map = df_annual_temp.set_index('annual_period')[value_col]
    
    # Tính tổng/trung bình nội suy theo năm
    agg_quarterly = df_quarterly_interpolated.groupby('annual_period')['interpolated_value'].agg(agg_func)
    
    # Tạo dataframe điều chỉnh
    df_annual_adjustment = pd.DataFrame({
        'annual_value': annual_map,
        'agg_interpolated': agg_quarterly
    }).dropna()
    
    print(f"  Số năm có dữ liệu: {len(df_annual_adjustment)}")
    
    # 3. TÍNH HỆ SỐ ĐIỀU CHỈNH
    if agg_func == 'sum':
        # Với Sum: adjustment là hệ số nhân (ratio)
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] / 
            df_annual_adjustment['agg_interpolated'].replace(0, np.nan)
        )
    else:  # mean
        # Với Mean: adjustment là chênh lệch (difference)
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] - 
            df_annual_adjustment['agg_interpolated']
        )
    
    # 4. HỢP NHẤT VÀ ĐIỀU CHỈNH
    df_quarterly_result = df_quarterly_interpolated.reset_index().copy()
    df_quarterly_result['annual_period'] = df_quarterly_result['annual_period'].astype(str)
    
    adj_reset = df_annual_adjustment['adjustment'].reset_index()
    adj_reset['annual_period'] = adj_reset['annual_period'].astype(str)
    
    df_quarterly_result = pd.merge(
        df_quarterly_result,
        adj_reset[['annual_period', 'adjustment']],
        on='annual_period',
        how='left'
    )
    
    # Áp dụng điều chỉnh
    if agg_func == 'sum':
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] * 
            df_quarterly_result['adjustment']
        )
    else:  # mean
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] + 
            df_quarterly_result['adjustment']
        )
    
    # 5. LỌC KẾT QUẢ VÀ CĂN CHỈNH INDEX
    result_series = df_quarterly_result.set_index('date')['quarterly_value'].dropna()
    
    # Lấy phạm vi năm từ dữ liệu gốc
    first_year = df_annual.index.min().year
    last_year = df_annual.index.max().year
    
    print(f"  Phạm vi năm: {first_year} - {last_year}")
    print(f"  Số quý trước lọc: {len(result_series)}")
    
    # Lọc chỉ giữ dữ liệu trong phạm vi năm hợp lệ
    result_series = result_series[
        (result_series.index.year >= first_year) & 
        (result_series.index.year <= last_year)
    ]
    
    print(f"  Số quý sau lọc: {len(result_series)}")
    
    return result_series

# --- ÁP DỤNG VÀ IN KẾT QUẢ ---

results = {}
for name, params in DISAGG_PARAMS.items():
    print(f"\n{'='*60}")
    print(f"Đang xử lý: {name}")
    print(f"{'='*60}")
    results[f'{name}_quarterly'] = benchmarked_linear_disaggregation(
        data_frames[name].copy(), 
        agg_func=params['agg_func']
    )

# Kết hợp tất cả dữ liệu
output_df = pd.concat(results.values(), axis=1, join='outer')
output_df.columns = results.keys()

# Lưu kết quả
output_df.to_csv('disaggregated_quarterly_data_fixed.csv')

print("\n" + "="*60)
print("Bảng Dữ liệu Quý được Nội suy (8 Quý Đầu):")
print("="*60)
print(output_df.head(8))

print("\n" + "="*60)
print("Bảng Dữ liệu Quý được Nội suy (8 Quý Cuối):")
print("="*60)
print(output_df.tail(8))

print(f"\nTổng số hàng: {len(output_df)}")
if len(output_df) > 0:
    print(f"Khoảng thời gian: {output_df.index.min()} đến {output_df.index.max()}")
else:
    print("❌ Không có dữ liệu. Kiểm tra debug output ở trên.")


Đang xử lý: fdi_inflow
  Index gốc (đầu): [Timestamp('1960-12-31 00:00:00'), Timestamp('1961-12-31 00:00:00'), Timestamp('1962-12-31 00:00:00')]
  Index gốc (cuối): [Timestamp('2022-12-31 00:00:00'), Timestamp('2023-12-31 00:00:00'), Timestamp('2024-12-31 00:00:00')]


ValueError: YS-JAN is not supported as period frequency